In [3]:
pip install tensorflow
pip install scipy

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.13.0-cp311-cp311-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.13.0-cp311-cp311-win_amd64.whl (276.6 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
                                              0.0/430.4 kB ? eta -:--:--
     --------                                92.2/430.4 kB 2.6 MB/s eta 0:00:01
     ---------------                        174.1/430.4 kB 2.6 MB/s eta 0:00:01
     -------------------------              286.7/430.4 kB 2.2 MB/s eta 0:00:01
     -------------------------------------  430.1

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import os
import cv2
import numpy as np
import random
import shutil
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Bidirectional, LSTM, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Define function to split data into train and val directories
def split_data(source, train, val, split_size):
    files = os.listdir(source)
    random.shuffle(files)
    split_index = int(split_size * len(files))
    train_files = files[:split_index]
    val_files = files[split_index:]

    for file in train_files:
        shutil.copy(os.path.join(source, file), os.path.join(train, file))

    for file in val_files:
        shutil.copy(os.path.join(source, file), os.path.join(val, file))

# Define function to process images
def process_images(from_dir, to_dir):
    os.makedirs(to_dir, exist_ok=True)

    # Process images and copy them to the target directory
    for file in os.listdir(from_dir):
        hand_img = cv2.imread(os.path.join(from_dir, file))
        if hand_img is not None:
            hand_img = cv2.resize(hand_img, (100, 100))  # Resize to smaller image size
            cv2.imwrite(os.path.join(to_dir, file), hand_img)


# Define paths
source_dir = 'processed_data'
train_dir = 'train'
val_dir = 'val'

# Create train and validation directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Split and process data
for label in os.listdir(source_dir):
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(val_dir, label), exist_ok=True)
    split_data(os.path.join(source_dir, label), os.path.join(train_dir, label), os.path.join(val_dir, label),
               split_size=0.8)
    process_images(os.path.join(train_dir, label), os.path.join(train_dir, label))
    process_images(os.path.join(val_dir, label), os.path.join(val_dir, label))

# Set up the data generators with resized images
img_size = (100, 100)  # Smaller image size
batch_size = 40  # Batch size

# Number of classes (replace with the actual number of classes in your dataset)
n_classes = len(os.listdir(train_dir))

train_datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_datagen = ImageDataGenerator(rescale=1. / 255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Define the model
model = Sequential()

# Add CNN layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))  # Additional Conv2D layer
model.add(MaxPooling2D((2, 2)))  # Additional MaxPooling2D layer
model.add(Flatten())

# Add Bidirectional LSTM layer
model.add(Reshape((1, -1)))  # You might need to modify this depending on your dataset
model.add(Bidirectional(LSTM(64)))

# Add a dense layer
model.add(Dense(128, activation='relu'))

# Add the output layer
model.add(Dense(n_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size)

# Save the trained model
model.save('model1.keras')



Found 3425 images belonging to 36 classes.
Found 2026 images belonging to 36 classes.
Epoch 1/5
85/85 [==============================] - 70s 771ms/step - loss: 1.4291 - accuracy: 0.6901 - val_loss: 0.0723 - val_accuracy: 0.9955
Epoch 2/5
85/85 [==============================] - 40s 474ms/step - loss: 0.0311 - accuracy: 0.9997 - val_loss: 0.0125 - val_accuracy: 1.0000
Epoch 3/5
85/85 [==============================] - 41s 488ms/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 4/5
85/85 [==============================] - 39s 462ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 5/5
85/85 [==============================] - 35s 414ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 1.0000
